In [26]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports
import GEMS_TCO
from GEMS_TCO import kernels
from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2

In [9]:
from pathlib import Path
# Assuming 'config' and 'load_data' class are defined and imported elsewhere

# --- Parameters derived from your framework ---
v: float = 0.5
space: List[str] = ['4', '4']
days: List[str] = ['0', '31']
mm_cond_number: int = 20
# --- End of framework parameters ---

lat_lon_resolution = [int(s) for s in space]
days_s_e = [int(d) for d in days]
days_list = list(range(days_s_e[0], days_s_e[1]))

# These values were not in the framework, so they remain as set in your snippet
years = ['2024']
month_range = [7] 

# Assuming 'config' is available in your environment
output_path = input_path = Path(config.mac_estimates_day_path)

## load ozone data from amarel
data_load_instance = load_data2(config.mac_data_load_path)

# Call the function using the variables from the framework
df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      # <-- Add this
lon_range=[123.0, 133.0]   # <-- Add this
)

Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


In [10]:
df_day_aggregated_list = []
df_day_map_list = []
for i in range(31):
    idx_for_datamap = [i*8, (i+1)*8]
    cur_map, cur_df = analysis_map_no_mm, agg_data_no_mm = data_load_instance.load_working_data(
    df_map, 
    idx_for_datamap, 
    ord_mm=None,  # or just omit it
    dtype=torch.float # or just omit it
)
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

print(df_day_aggregated_list[0].shape)


torch.Size([8960, 4])


In [11]:
instance1 = kernels.vecchia_experiment(0.5, df_day_map_list[0], df_day_aggregated_list[0], nns_map, mm_cond_number, nheads=10)

a = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769]
#a = [30.2594, 0.665, 1.8981, 0.0, 0.1317, -0.0, 1.9785]
#a = [45.1402, 0.6299, 0.7308, -0.0003, -0.0151, 0.0, 7.8922]
#a = [21.7335, 1.2817, 1.5946, 0.042, -0.1241, 0.218, 4.8654]
#a = [20.453542336448137, 1.4506118600616982, 2.43096923637867, -0.03476556019978718, -0.1559262606484541, 0.1254833595232136, 3.938183829354925]
params = torch.tensor(a, dtype=torch.float64, requires_grad=True)

cov_map = instance1.cov_structure_saver(params, instance1.matern_cov_anisotropy_v05)  
instance1.vecchia_oct22( params, instance1.matern_cov_anisotropy_v05, cov_map )



tensor(15995.6299, grad_fn=<AddBackward0>)

v = 0.5 # smooth
mm_cond_number = 20
nheads = 20
lr = 0.01
step = 80
gamma_par = 0.9
epochs = 500

In [12]:
v = 0.5 # smooth
mm_cond_number = 20
nheads = 300
lr = 0.02
step = 100
gamma_par = 0.3
epochs = 900

# fit 8 hours of data

In [ ]:
# ------------------------------------
# Main Optimization Loop (Updated)
# ------------------------------------

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
df_day_aggregated_list = []
df_day_map_list = []
num_days_to_load = 31 # From reference code

for i in range(num_days_to_load):
    idx_for_datamap = [i*8, (i+1)*8]
    # Using the new load function from the reference code
    cur_map, cur_df = data_load_instance.load_working_data(
        df_map, 
        idx_for_datamap, 
        ord_mm=None,
        # Using float64 to ensure compatibility with kernels
        dtype=torch.float64 
    )
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

print(f"Data loaded for {len(df_day_map_list)} days.")
# print(df_day_aggregated_list[0].shape) # From reference code

# --- 2. Run optimization loop over pre-loaded data ---

# This list is now just for iterating
#days_list = range(len(df_day_map_list)) 
days_list = [0]
for day in days_list:  
    
    # Get the pre-loaded data for this day
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]

    a = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769]
    a = [28, 0.65, 0.75, 0.022, -0.144, 0.198, 2.5]
    params = torch.tensor(a, dtype=torch.float64, requires_grad=True)
    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params.detach().numpy()}')
            
    # --- Data loading is now done *before* the loop ---

    # We need to define the device (though we aren't passing it anymore)
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
            # device = device_str  <--- REMOVED: This was causing the TypeError
        )

    start_time = time.time()
    # Adjusted optimizer call based on expected return values (step size changed to step)
    optimizer, scheduler = model_instance.optimizer_fun(
        params, 
        lr=lr, 
        betas=(0.9, 0.8), 
        eps=1e-8, 
        step_size=step, # Using the 'step' variable here
        gamma=gamma_par  # Using gamma_par
    ) 

    # --- CRITICAL CORRECTION ---
    # 1. We no longer need to create a separate 'instance_map'.
    #    'model_instance' is already the correct instance.
    # 2. We do NOT pre-calculate 'cov_map'. The optimized training loop
    #    'run_vecc_may9' does this internally on each epoch
    #    to ensure the gradients are correct.
    # 3. We call 'run_vecc_may9' (the optimized loop) instead of 'run_vecc_grp9'.
    #    This version does not take 'cov_map' as an argument.
    
    # Calling the optimized 'run_vecc_may9'
    out, epoch_ran = model_instance.run_vecc_oct22(
        params, 
        optimizer,
        scheduler, 
        model_instance.matern_cov_anisotropy_v05, 
        epochs=epochs
    )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Day {day+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day+1} final results: {out}")

print("\n--- All Days Processed ---")


# 5000 takes 47 mn
# Day 1 final results: [29.7507942474575, 0.9895617352853928, 1.0668105024738395, 0.03646879682204495, -0.15565418931819383, 0.17944566056749772, 1.8907510307884106, 64567.25091631662]
# 1250 take 10mn
#  29.89872140748479, 1.1529714369768411, 1.7862356661866714, 0.03927727761515986, -0.15656505052873793, 0.1320573870050866, 4.130349222670235, 15749.646109280728

# try just fitting one hour

In [ ]:


# ------------------------------------
# Main Optimization Loop (Updated)
# ------------------------------------

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
df_day_aggregated_list = []
df_day_map_list = []
num_days_to_load = 31 # From reference code

for i in range(num_days_to_load):
    idx_for_datamap = [i*8, i*8+1]
    # Using the new load function from the reference code
    cur_map, cur_df = data_load_instance.load_working_data(
        df_map, 
        idx_for_datamap, 
        ord_mm=None,
        # Using float64 to ensure compatibility with kernels
        dtype=torch.float64 
    )
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

print(f"Data loaded for {len(df_day_map_list)} days.")
# print(df_day_aggregated_list[0].shape) # From reference code

# --- 2. Run optimization loop over pre-loaded data ---

# This list is now just for iterating
#days_list = range(len(df_day_map_list)) 
days_list = [0]
for day in days_list:  
    
    # Get the pre-loaded data for this day
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]

    
    #a = [36.75, 0.98, 1.06, 0, 0, 0, 1.890]
    #params = torch.tensor(a, dtype=torch.float64, requires_grad=True)

    a = [22, 0.98, 1.06, 0, 0, 0, 1.890]
    a = [28.75, 0.98, 1.06, 0.036, -0.155, 0.179, 1.890]
    a = [28.75, 0.98, 1.06, 0, 0, 0, 1.890]
    #a = [21, 1, 1.5, 0,0,0,4.769]
    # NEW: Define params as a list of 1-element tensors
    # This is required to group them for the optimizer.
    params_list = [
        torch.tensor([val], dtype=torch.float64, requires_grad=True) for val in a
    ]

    

    # NEW: Define learning rates and groups exactly as in your example
    lr_slow, lr_fast = 0.02, 0.02
    slow_indices = [ 1, 2, 3,4,5,6]
    fast_indices = [0]

    param_groups = [
        {'params': [params_list[idx] for idx in slow_indices], 'lr': lr_slow, 'name': 'slow_group'},
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
            
    # --- Data loading is now done *before* the loop ---

    # We need to define the device (though we aren't passing it anymore)
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
            # device = device_str  <--- REMOVED: This was causing the TypeError
        )

    start_time = time.time()
    # Adjusted optimizer call based on expected return values (step size changed to step)
    optimizer, scheduler = model_instance.optimizer_fun10_23(
            param_groups,     # <--- Pass the new param_groups list
            lr=lr,            # <--- This is now just a default
            betas=(0.9, 0.8), 
            eps=1e-8, 
            step_size=step, 
            gamma=gamma_par
        )

    # --- CRITICAL CORRECTION ---
    # 1. We no longer need to create a separate 'instance_map'.
    #    'model_instance' is already the correct instance.
    # 2. We do NOT pre-calculate 'cov_map'. The optimized training loop
    #    'run_vecc_may9' does this internally on each epoch
    #    to ensure the gradients are correct.
    # 3. We call 'run_vecc_may9' (the optimized loop) instead of 'run_vecc_grp9'.
    #    This version does not take 'cov_map' as an argument.
    
    # Calling the optimized 'run_vecc_may9'
    out, epoch_ran = model_instance.run_vecc_oct23(
            params_list,     # <--- Pass the new params_list
            optimizer,
            scheduler, 
            model_instance.matern_cov_anisotropy_v05, 
            epochs=epochs
        )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Day {day+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day+1} final results: {out}")

print("\n--- All Days Processed ---")

Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 1250.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [tensor([28.7500], dtype=torch.float64, requires_grad=True), tensor([0.9800], dtype=torch.float64, requires_grad=True), tensor([1.0600], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([1.8900], dtype=torch.float64, requires_grad=True)]
Converged at epoch 701
Epoch 702,  
 vecc Parameters: [ 2.81337847e+01  9.07524208e-01  1.81091766e+00 -4.03066803e-06
  5.84709994e-06  0.00000000e+00  1.51297605e+00]
FINAL STATE: Epoch 702, Loss: 2087.696500414076, 
 vecc Parameters: [28.133784739040472, 0.9075242075841814, 1.8109176606611543, -4.030668029920314e-06, 5.847099939599732e-06, 0.0, 1.512976047183501]
Day 1 optimization finished in 155.04s over 702 epoch

# fit one hour old version

In [19]:
# ------------------------------------
# Main Optimization Loop (Updated)
# ------------------------------------

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
df_day_aggregated_list = []
df_day_map_list = []
num_days_to_load = 31 # From reference code

for i in range(num_days_to_load):
    idx_for_datamap = [i*8, i*8+1]
    # Using the new load function from the reference code
    cur_map, cur_df = data_load_instance.load_working_data(
        df_map, 
        idx_for_datamap, 
        ord_mm=None,
        # Using float64 to ensure compatibility with kernels
        dtype=torch.float64 
    )
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

print(f"Data loaded for {len(df_day_map_list)} days.")
# print(df_day_aggregated_list[0].shape) # From reference code

# --- 2. Run optimization loop over pre-loaded data ---

# This list is now just for iterating
#days_list = range(len(df_day_map_list)) 
days_list = [0]
for day in days_list:  
    
    # Get the pre-loaded data for this day
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]

    a = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769]
    a = [21.303, 1.307, 1.563, 0, 0, 0, 4.769]
    params = torch.tensor(a, dtype=torch.float64, requires_grad=True)
    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params.detach().numpy()}')
            
    # --- Data loading is now done *before* the loop ---

    # We need to define the device (though we aren't passing it anymore)
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
            # device = device_str  <--- REMOVED: This was causing the TypeError
        )

    start_time = time.time()
    # Adjusted optimizer call based on expected return values (step size changed to step)
    optimizer, scheduler = model_instance.optimizer_fun(
        params, 
        lr=lr, 
        betas=(0.9, 0.8), 
        eps=1e-8, 
        step_size=step, # Using the 'step' variable here
        gamma=gamma_par  # Using gamma_par
    ) 

    # --- CRITICAL CORRECTION ---
    # 1. We no longer need to create a separate 'instance_map'.
    #    'model_instance' is already the correct instance.
    # 2. We do NOT pre-calculate 'cov_map'. The optimized training loop
    #    'run_vecc_may9' does this internally on each epoch
    #    to ensure the gradients are correct.
    # 3. We call 'run_vecc_may9' (the optimized loop) instead of 'run_vecc_grp9'.
    #    This version does not take 'cov_map' as an argument.
    
    # Calling the optimized 'run_vecc_may9'
    out, epoch_ran = model_instance.run_vecc_oct22(
        params, 
        optimizer,
        scheduler, 
        model_instance.matern_cov_anisotropy_v05, 
        epochs=epochs
    )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Day {day+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day+1} final results: {out}")

print("\n--- All Days Processed ---")

Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 1250.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [21.303  1.307  1.563  0.     0.     0.     4.769]
Converged at epoch 230
Epoch 231,  
 vecc Parameters: [ 2.20562660e+01  2.10108999e+00  3.29873099e+00 -2.36777772e-06
 -2.32200519e-07  0.00000000e+00  4.00362099e+00]
FINAL STATE: Epoch 231, Loss: 2036.6417848718395, 
 vecc Parameters: [22.056265955285376, 2.101089989181403, 3.298730986510232, -2.3677777249201222e-06, -2.3220051907190966e-07, 0.0, 4.003620989283949]
Day 1 optimization finished in 52.15s over 231 epochs.
Day 1 final results: [22.056265955285376, 2.101089989181403, 3.298730986510232, -2.3677777249201222e-06, -2.3220051907190966e-07, 0.0, 4.003620989283949, 2036.6417848718395]

--- All Days Processed ---


## dont use below above is much faster

In [16]:
# --- 2. Run optimization loop over pre-loaded data (IMPROVED) ---
days_list = [0] # Just run for the first day
for day in days_list:  
    
    # Get the pre-loaded data for this day
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]

    # --- Initial parameters in NATURAL scale (STABLE) ---
    a_natural = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769]
    # NOTE: The log_indices are still useful for *printing* in the advanced loop,
    # but we will no longer use the log-transformation for initialization.
    log_indices = [0, 1, 2, 6] 
    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number}')
            
    # --- Determine device ---
    # Moved device logic up for clarity, but keep the check
    device_str = 'cpu'
    if torch.cuda.is_available():
        try:
            torch.tensor([1.0]).to('cuda')
            device_str = 'cuda'
            print("Using device: cuda")
        except Exception as e:
            # Added import check for warnings
            import warnings
            warnings.warn(f"CUDA available but failed to initialize. {e}. Defaulting to CPU.")
            device_str = 'cpu'
            print("Using device: cpu")
    else:
        print("Using device: cpu")
    
    
    # ====================================================================
    # --- RUN: Advanced Loop with Stable Setup ---
    # ====================================================================
    print(f"\n--- RUN: Using 'run_vecc_advanced' with STABLE StepLR ---")
    
    # Initialize params using NATURAL SCALE directly
    # This avoids the numerical instability from taking the log of near-zero values.
    # The advanced loop's printing function will convert back to natural scale.
    params_stable = torch.tensor(a_natural, dtype=torch.float64, requires_grad=True).to(device_str)
    print(f'Initial parameters (natural scale): \n {params_stable.detach().cpu().numpy().round(4)}')
    
    # --- Model Initialization ---
    # (Identical to previous code)
    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads,
        )

    start_time = time.time()
    
    # --- OPTIMIZER: Use StepLR for stability ---
    optimizer, scheduler = model_instance.optimizer_fun(
        params_stable, 
        scheduler_type='step', # Explicitly use the 'step' scheduler
        lr=lr, 
        betas=(0.9, 0.8), 
        eps=1e-8,
        step_size=step, # Use step=80 from your configuration
        gamma=gamma_par # Use gamma_par=0.1 from your configuration
    ) 
    
    # Calling the advanced 'run_vecc_advanced' loop
    # NOTE: The printing in this function needs to be checked. If parameters are
    # NOT log-transformed, the printing function (get_printable_params) will
    # apply 'torch.exp' to the log_indices, which is now incorrect.
    #
    # Assuming 'run_vecc_advanced' is modified to correctly handle natural-scale
    # parameters when log_param_indices is passed but the input is not log-space.
    # --- CRITICAL ASSUMPTION ---
    # Since you want to use natural scale, we should NOT pass log_param_indices
    # unless you explicitly want to apply constraints/transforms inside the loop.
    # Since the simpler approach worked, let's revert to the minimal stable setup.
    #
    
    # *** REVISED: Calling the STABLE 'run_vecc_oct22' loop ***
    # This is the approach that worked, but with your desired high epoch count
    out, epoch_ran = model_instance.run_vecc_oct22(
        params_stable, # Use natural-scale params
        optimizer,
        scheduler, 
        model_instance.matern_cov_anisotropy_v05, 
        epochs=epochs # Use epochs=300
    )

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"--- 'run_vecc_oct22' (Stable Setup) finished in {epoch_time:.2f}s over {epoch_ran+1} epochs ---")
    print(f"Final results: {out}")

print("\n--- All Days Processed ---")


--- Starting Day 1 (2024-07-1) ---
Data size per day: 1250.0, smooth: 0.5
mm_cond_number: 20
Using device: cpu

--- RUN: Using 'run_vecc_advanced' with STABLE StepLR ---
Initial parameters (natural scale): 
 [21.303  1.307  1.563  0.022 -0.144  0.198  4.769]
FINAL STATE: Epoch 500, Loss: 16717.15700508694, 
 vecc Parameters: [18.592316201664097, 1.4125517238033354, 1.2647947634869554, -0.10675469109734659, -0.13885701275772608, 0.2199784453104275, 1.3479833641217873]
--- 'run_vecc_oct22' (Stable Setup) finished in 899.35s over 500 epochs ---
Final results: [18.592316201664097, 1.4125517238033354, 1.2647947634869554, -0.10675469109734659, -0.13885701275772608, 0.2199784453104275, 1.3479833641217873, 16717.15700508694]

--- All Days Processed ---
